In [1]:
!pip install tensorboardX

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import copy
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #三通道输入，十输出，2*2池化，2卷积3fc
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg

#CIFAR-10数据集包含10个类别，每个类别有6000张32x32的彩色图像。
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 100 #单次批次大小
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#以下用于直接学，就嗯学
#定义神经网络
device='cuda:0'
net = Net().to(device)  # Move the model to GPU
net.zero_grad()  # zeroes the gradient buffers of all parameters
print(net)

#定义损失函数
criterion = nn.CrossEntropyLoss().to(device)

#定义更新函数（SGD）
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#测试输入集{input,target}
#input = torch.randn(1, 1, 32, 32).to(device)  # Move input tensor to GPU
#target = torch.randn(10).to(device)  # a dummy target, for example
#target = target.view(1, -1)  # make it the same shape as output

#前向传播并计算损失
#output = net(input)
#print(output)
#loss = criterion(output, target)
#print(loss)

#清除优化器累计梯度更新
#optimizer.zero_grad()   # zero the gradient buffers

#更新模型
#loss.backward()   #后向传播计算梯度
#optimizer.step()    # 更新模型参数

#载入训练集
#特征，标签 特征为图片集，标签为各图片集对应class在classes中的位置
dataiter = iter(trainloader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

#训练神经网络
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
print('Finished Training')

#保存训练好的神经网络模型
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
#以下用于测试联邦学习
class main_node():
  def __init__(self,mydevice='cpu',myglepoch=3,mynodenum=3,mysubepoch=3):
    self.device=mydevice
    self.net=Net()
    self.criterion = nn.CrossEntropyLoss()
    if self.device!='cpu':
      self.net=self.net.to(self.device)
      self.criterion = self.criterion.to(self.device)
    self.optimizer = optim.SGD(self.net.parameters(), lr=0.001, momentum=0.9)
    self.global_epoch=myglepoch
    self.node_num=mynodenum
    self.sub_epoch=mysubepoch
  def federated_train(self):
    train_loss, train_accuracy = [], []
    for global_epoch_i in range(self.global_epoch):  #全局轮
      print('Current Global Epoch: '+str(global_epoch_i))
      global_epoch_weights, global_epoch_local_losses = [], []
      for sub_id in range(self.node_num): #子节点
        print('Current Sub_id: '+str(sub_id))
        sub=sub_node(copy.deepcopy(self.net),mydevice='cuda:0')
        subweights,subloss=sub.train_net(trainloader,self.sub_epoch)
        global_epoch_weights.append(copy.deepcopy(subweights))
        global_epoch_local_losses.append(copy.deepcopy(subloss))
      #全局轮次更新
      self.net.load_state_dict(average_weights(global_epoch_weights))
      loss_avg = sum(global_epoch_local_losses) / len(global_epoch_local_losses)
      train_loss.append(loss_avg)
class sub_node(main_node):
  def __init__(self,mynet,mydevice='cpu'):
    self.device=mydevice
    self.net=mynet
    self.criterion = nn.CrossEntropyLoss()
    if self.device!='cpu':
      self.net=self.net.to(self.device)
      self.criterion = self.criterion.to(self.device)
    self.optimizer = optim.SGD(self.net.parameters(), lr=0.001, momentum=0.9)

  def train_net(self,trainset,subepoch):
    epoch_loss = []
    for epoch_i in range(subepoch):  #本地轮
      batch_loss = []
      print('Current Sub Epoch: '+str(epoch_i))
      for batch_i, (images, labels) in enumerate(trainset):   #对于各批次
        if self.device!='cpu':
          images, labels = images.to(self.device), labels.to(self.device)
        self.optimizer.zero_grad() #FL案例里清零的是net，一想一个不吱声
        outputs=self.net(images)
        loss = self.criterion(outputs, labels)  #计算更新
        loss.backward() #后向传播计算梯度
        self.optimizer.step() #根据梯度修正参数
        batch_loss.append(loss.item())
      epoch_loss.append(sum(batch_loss)/len(batch_loss))
    return self.net.state_dict(), sum(epoch_loss) / len(epoch_loss) #回传参数和损失

rootNode=main_node(mydevice='cuda:0',myglepoch=5,mysubepoch=5)
rootNode.federated_train()
#保存训练好的神经网络模型
PATH = './cifar_net.pth'
torch.save(rootNode.net.state_dict(), PATH)

In [ ]:
#以下代码用于异步多线程执行联邦学习(实测没有加速效果，计算密集度太高)
import threading

class main_node():
  def __init__(self,mydevice='cpu',myglepoch=3,mynodenum=3,mysubepoch=3):
    self.device=mydevice
    self.net=Net()
    self.criterion = nn.CrossEntropyLoss()
    if self.device!='cpu':
      self.net=self.net.to(self.device)
      self.criterion = self.criterion.to(self.device)
    self.optimizer = optim.SGD(self.net.parameters(), lr=0.001, momentum=0.9)
    self.global_epoch=myglepoch
    self.node_num=mynodenum
    self.sub_epoch=mysubepoch

  def federated_train(self):
    train_loss, train_accuracy = [], []
    for global_epoch_i in range(self.global_epoch):  #全局轮
      print('Current Global Epoch: '+str(global_epoch_i))
      global_epoch_weights, global_epoch_local_losses = [], []
      threads = []
      for sub_id in range(self.node_num): #子节点
        sub=sub_node(copy.deepcopy(self.net),sub_id,mydevice='cuda:0')
        thread = threading.Thread(target=self.train_sub_node, args=(sub, trainloader, self.sub_epoch, global_epoch_weights, global_epoch_local_losses))
        threads.append(thread)
        thread.start()
      for thread in threads:
        thread.join()
      #全局轮次更新
      self.net.load_state_dict(average_weights(global_epoch_weights))
      loss_avg = sum(global_epoch_local_losses) / len(global_epoch_local_losses)
      train_loss.append(loss_avg)

  def train_sub_node(self, sub, trainloader, subepoch, global_epoch_weights, global_epoch_local_losses):
    subweights,subloss=sub.train_net(trainloader,subepoch)
    global_epoch_weights.append(copy.deepcopy(subweights))
    global_epoch_local_losses.append(copy.deepcopy(subloss))

class sub_node(main_node):
  def __init__(self,mynet,mysubid,mydevice='cpu'):
    self.device=mydevice
    self.sub_id=mysubid
    self.net=mynet
    self.criterion = nn.CrossEntropyLoss()
    if self.device!='cpu':
      self.net=self.net.to(self.device)
      self.criterion = self.criterion.to(self.device)
    self.optimizer = optim.SGD(self.net.parameters(), lr=0.001, momentum=0.9)

  def train_net(self,trainset,subepoch):
    epoch_loss = []
    for epoch_i in range(subepoch):  #本地轮
      batch_loss = []
      print('Sub: '+str(self.sub_id)+' Epoch: '+str(epoch_i))
      for batch_i, (images, labels) in enumerate(trainset):   #对于各批次
        if self.device!='cpu':
          images, labels = images.to(self.device), labels.to(self.device)
        self.optimizer.zero_grad() #FL案例里清零的是net，一想一个不吱声
        outputs=self.net(images)
        loss = self.criterion(outputs, labels)  #计算更新
        loss.backward() #后向传播计算梯度
        self.optimizer.step() #根据梯度修正参数
        batch_loss.append(loss.item())
      epoch_loss.append(sum(batch_loss)/len(batch_loss))
    return self.net.state_dict(), sum(epoch_loss) / len(epoch_loss) #回传参数和损失

rootNode=main_node(mydevice='cuda:0',myglepoch=3,mynodenum=5,mysubepoch=3)
rootNode.federated_train()
#保存训练好的神经网络模型
PATH = './cifar_net.pth'
torch.save(rootNode.net.state_dict(), PATH)


In [ ]:
#以下用于从本地载入模型进行测试
#载入测试集
dataiter = iter(testloader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

#加载本地模型
device='cuda:0'
net = Net().to(device)
PATH = './cifar_net.pth'
net.load_state_dict(torch.load(PATH))
#推理测试
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1) #data中每个结果的最大值,data中每个结果的最大值索引
        total += labels.size(0) #本批次测试数量
        correct += (predicted == labels).sum().item() #本批次正确数量

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')